In [40]:
key = '6448425a457377653630797a536a4c'

# https://www.data.go.kr/data/15073855/openapi.do
import json
import requests
url = 'http://openapi.seoul.go.kr:8088/6448425a457377653630797a536a4c/json/parkingKickboard/1/5/'

response = requests.get(url)
content = response.content.decode('utf-8')
data = json.loads(content)
print(data)

{'parkingKickboard': {'list_total_count': 190, 'RESULT': {'CODE': 'INFO-000', 'MESSAGE': '정상 처리되었습니다'}, 'row': [{'SN': 1.0, 'SGG_NM': '종로구', 'PSTN': '팔판동 115-63', 'DTL_PSTN': '청와대 춘추문 맞은편 인근', 'STAND_YN': 'Y', 'STAND_SIZE': 6.0}, {'SN': 2.0, 'SGG_NM': '종로구', 'PSTN': '연건동 218-1', 'DTL_PSTN': 'KT광화문 혜화지사 앞', 'STAND_YN': 'Y', 'STAND_SIZE': 6.0}, {'SN': 3.0, 'SGG_NM': '종로구', 'PSTN': '연건동 178-3', 'DTL_PSTN': '홍익대학교 대학로 맞은편', 'STAND_YN': 'Y', 'STAND_SIZE': 6.0}, {'SN': 4.0, 'SGG_NM': '종로구', 'PSTN': '동승동 1-24', 'DTL_PSTN': '대학로 마로니에공원 앞', 'STAND_YN': 'Y', 'STAND_SIZE': 8.0}, {'SN': 5.0, 'SGG_NM': '종로구', 'PSTN': '와룡동 75-4', 'DTL_PSTN': '연악사 맞은편', 'STAND_YN': 'Y', 'STAND_SIZE': 8.0}]}}


In [41]:
def scoot_parklot_data(data):
    items = data['parkingKickboard']['row']
    scooter_info = []
    for item in items:
        info = {
            '순번': item.get('SN'),
            '시군구명': item.get('SGG_NM'),
            '주소': item.get('PSTN'),
            '상세위치': item.get('DTL_PSTN'),
            '거치대 유무': item.get('STAND_YN'),
            '거치대 크기': item.get('STAND_SIZE'),

        }
        scooter_info.append(info)
    return scooter_info

scooter_info = scoot_parklot_data(data)
print(scooter_info)
scooter_info[1]



[{'순번': 1.0, '시군구명': '종로구', '주소': '팔판동 115-63', '상세위치': '청와대 춘추문 맞은편 인근', '거치대 유무': 'Y', '거치대 크기': 6.0}, {'순번': 2.0, '시군구명': '종로구', '주소': '연건동 218-1', '상세위치': 'KT광화문 혜화지사 앞', '거치대 유무': 'Y', '거치대 크기': 6.0}, {'순번': 3.0, '시군구명': '종로구', '주소': '연건동 178-3', '상세위치': '홍익대학교 대학로 맞은편', '거치대 유무': 'Y', '거치대 크기': 6.0}, {'순번': 4.0, '시군구명': '종로구', '주소': '동승동 1-24', '상세위치': '대학로 마로니에공원 앞', '거치대 유무': 'Y', '거치대 크기': 8.0}, {'순번': 5.0, '시군구명': '종로구', '주소': '와룡동 75-4', '상세위치': '연악사 맞은편', '거치대 유무': 'Y', '거치대 크기': 8.0}]


{'순번': 2.0,
 '시군구명': '종로구',
 '주소': '연건동 218-1',
 '상세위치': 'KT광화문 혜화지사 앞',
 '거치대 유무': 'Y',
 '거치대 크기': 6.0}

In [42]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema import Document

documents = [Document(page_content=", ".join([f"{key}: {str(info[key])}" for key in ['순번','시군구명','주소','상세위치','거치대 유무','거치대 크기']])) for info in scooter_info]
print(documents)



[Document(metadata={}, page_content='순번: 1.0, 시군구명: 종로구, 주소: 팔판동 115-63, 상세위치: 청와대 춘추문 맞은편 인근, 거치대 유무: Y, 거치대 크기: 6.0'), Document(metadata={}, page_content='순번: 2.0, 시군구명: 종로구, 주소: 연건동 218-1, 상세위치: KT광화문 혜화지사 앞, 거치대 유무: Y, 거치대 크기: 6.0'), Document(metadata={}, page_content='순번: 3.0, 시군구명: 종로구, 주소: 연건동 178-3, 상세위치: 홍익대학교 대학로 맞은편, 거치대 유무: Y, 거치대 크기: 6.0'), Document(metadata={}, page_content='순번: 4.0, 시군구명: 종로구, 주소: 동승동 1-24, 상세위치: 대학로 마로니에공원 앞, 거치대 유무: Y, 거치대 크기: 8.0'), Document(metadata={}, page_content='순번: 5.0, 시군구명: 종로구, 주소: 와룡동 75-4, 상세위치: 연악사 맞은편, 거치대 유무: Y, 거치대 크기: 8.0')]


In [43]:
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
embedding_function = SentenceTransformerEmbeddings(model_name="jhgan/ko-sroberta-multitask")

c:\Users\AISW-203-102\study\myenv\lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [44]:
from langchain.vectorstores import FAISS
db = FAISS.from_documents(documents, embedding_function )

In [45]:
query = "종로구"
docs = db.similarity_search(query)
print(docs[0].page_content)
docs

순번: 4.0, 시군구명: 종로구, 주소: 동승동 1-24, 상세위치: 대학로 마로니에공원 앞, 거치대 유무: Y, 거치대 크기: 8.0


[Document(metadata={}, page_content='순번: 4.0, 시군구명: 종로구, 주소: 동승동 1-24, 상세위치: 대학로 마로니에공원 앞, 거치대 유무: Y, 거치대 크기: 8.0'),
 Document(metadata={}, page_content='순번: 5.0, 시군구명: 종로구, 주소: 와룡동 75-4, 상세위치: 연악사 맞은편, 거치대 유무: Y, 거치대 크기: 8.0'),
 Document(metadata={}, page_content='순번: 3.0, 시군구명: 종로구, 주소: 연건동 178-3, 상세위치: 홍익대학교 대학로 맞은편, 거치대 유무: Y, 거치대 크기: 6.0'),
 Document(metadata={}, page_content='순번: 2.0, 시군구명: 종로구, 주소: 연건동 218-1, 상세위치: KT광화문 혜화지사 앞, 거치대 유무: Y, 거치대 크기: 6.0')]

In [46]:
from langchain_community.chat_models import ChatOllama
llm = ChatOllama(model="gemma2:9b", temperature=0, base_url="http://127.0.0.1:11434/") #http://127.0.0.1:11434

In [47]:
from langchain.chains import RetrievalQA
question = """종로구 킥보드 주차구역 정보에 대해 알려줘"""
qa_chain = RetrievalQA.from_chain_type(llm,retriever=db.as_retriever())
result = qa_chain({"query": question})
print(result['result'])

종로구 내 킥보드 주차 가능 지역은 다음과 같습니다.

* **순번: 4.0**
    * 시군구명: 종로구
    * 주소: 동승동 1-24
    * 상세위치: 대학로 마로니에공원 앞
    * 거치대 유무: Y
    * 거치대 크기: 8.0

* **순번: 3.0**
    * 시군구명: 종로구
    * 주소: 연건동 178-3
    * 상세위치: 홍익대학교 대학로 맞은편
    * 거치대 유무: Y
    * 거치대 크기: 6.0

* **순번: 5.0**
    * 시군구명: 종로구
    * 주소: 와룡동 75-4
    * 상세위치: 연악사 맞은편
    * 거치대 유무: Y
    * 거치대 크기: 8.0

* **순번: 2.0**
    * 시군구명: 종로구
    * 주소: 연건동 218-1
    * 상세위치: KT광화문 혜화지사 앞
    * 거치대 유무: Y
    * 거치대 크기: 6.0





In [51]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnableMap

template = """
너는 서울시 전동킥보드 주차구역 안내로봇이야. 
상세위치를 알려주면서 가장 가까운 지하철역까지 안내해줘야해
Answer the question as based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [52]:
retriever = db.as_retriever(search_type="similarity", search_kwargs={'k':10, 'fetch_k': 100})

In [53]:
from langchain.schema.runnable import RunnableMap
chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x['question']),
    "question": lambda x: x['question']
}) | prompt | llm

In [54]:
from IPython.display import Markdown
Markdown(chain.invoke({'question': "종로구 킥보드 어디에 주차하면 되는지 알려줘"}).content)

C:\Users\AISW-203-102\AppData\Local\Temp\ipykernel_34772\121337404.py:3: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  "context": lambda x: retriever.get_relevant_documents(x['question']),


종로구에서 전동킥보드를 주차할 수 있는 곳은 다음과 같습니다. 

* **대학로 마로니에공원 앞** (동승동 1-24) -  가까운 지하철역: **혜화역**
* **연악사 맞은편** (와룡동 75-4) - 가까운 지하철역: **안국역**
* **홍익대학교 대학로 맞은편** (연건동 178-3) - 가까운 지하철역: **혜화역**
* **청와대 춘추문 맞은편 인근** (팔판동 115-63) - 가까운 지하철역: **경복궁역**
* **KT광화문 혜화지사 앞** (연건동 218-1) - 가까운 지하철역: **혜화역**


어떤 곳이 가장 편리한지는 주차 위치와 지하철 이용 계획에 따라 달라질 수 있습니다.  
